In [1]:
#Libraries Used
import socket
import pickle
import time

In [2]:
#Client 2 has its public and private keys and public key of PKDA
with open('publicKey_client2.pkl','rb') as file:
    publicKey_client2 = pickle.load(file)
    
with open('privateKey_client2.pkl','rb') as file:
    privateKey_client2 = pickle.load(file)
    
with open('publicKey_PKDA.pkl','rb') as file:
    publicKey_PKDA = pickle.load(file)

In [3]:
#Evaluating the binary exponention 
#M^e (mod n)
#C^d (mod n)

def binary_modular_exponention(m,e,n):
    result = 1
    m = m % n
    if (m == 0):
        return 0
    while(e>0):
        if(e&1):
            result = (result*m) % n
        e = e>>1
        m = (m*m) % n
    return result

In [4]:
#Encryption Algorithm
def encrypt_data(encryption_key, message):
    cipher_text = ''
    for value in range(0, len(message)):
        encrypted_char = binary_modular_exponention(ord(message[value]), encryption_key[0], encryption_key[1])
        cipher_text += chr(encrypted_char)
    return cipher_text

In [5]:
#Decryption Algorithm
def decrypt_data(decryption_key, cipher_data):
    original_text = ''
    for value in range(0, len(cipher_data)):
        decrypted_char = binary_modular_exponention(ord(cipher_data[value]), decryption_key[0], decryption_key[1])
        original_text += chr(decrypted_char)
    return original_text

In [6]:
#Closing any previous socket connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.close()

In [7]:
#Defining the local host and different ports
host = 'local host'
server_port = 5000
client1_port = 5001
client2_port = 5002

#Defining the Socket Connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.bind(('', client1_port))
s.listen(5)

#Conecting to Client 1
print('Waiting for connection')
c, addr = s.accept()
print("CONNECTION FROM:", str(addr))
c.send(b"Connected Succcessfully to Client2")
print()

#Receiving message send from Step 3
msg = c.recv(1024)
decrypted_text = decrypt_data(privateKey_client2, msg.decode())

print('Message from Client 1(after decryption) : ',decrypted_text)
print()
#n1 from Client 1
n1 = decrypted_text.split('||')[1]
c.close()
s.close()

#Connection established with Server
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.connect(('127.0.0.1', server_port))
msg = s.recv(1024)
print(msg.decode())
print()

#Step 4 Sending Request to Server PKDA
#(4) Request || T2
t2 = time.time()
request = decrypted_text.split('||')[0]
request_ts = request + '||' + str(t2)
s.send(request_ts.encode())
print('Request sent to server : ',request_ts)
print()

#Step 5 Receiving message from Server
#(5) E(PRauth, [PUa || Request || T2])
msg = s.recv(1024)
#Decode the message receieved from PKDA from Step 5
decrypted_text_2 = decrypt_data(publicKey_PKDA, msg.decode())
print('Response received from server(after decryption) \n : ',decrypted_text)
print()
#Finding the public key of client to be connected
requested_public_key_e = int(decrypted_text_2.split('||')[0])
requested_public_key_n = int(decrypted_text_2.split('||')[1])
requested_public_key = [requested_public_key_e,requested_public_key_n]
print()
print('Public key of requested client :',requested_public_key)
print()
print('Sent Time Stamp T2: ', decrypted_text_2.split('||')[3])
print()
print('Recieved Time Stamp T2: ', str(time.time()))
print()
s.close()

#Step 6 Sending Request to Client 1
#(6) E(PUa, [ N1 || N2])
n2 = str(time.time())
n1_n2 = n1 + '||' + n2
encrypted_text = encrypt_data(requested_public_key, n1_n2)
print('Message sending from Client 2(this client) to Client 1 : ', n1_n2)

s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.connect(('127.0.0.1', client2_port))
msg = s.recv(1024)
print(msg.decode())
s.send(encrypted_text.encode())
print()

#Receiving message from Step 7 sent from Client 1
msg = s.recv(1024)
decrypted_text = decrypt_data(privateKey_client2, msg.decode())
print('Client 1 response : ', decrypted_text)

if(decrypted_text == n2):
    print('Original n2 =',n2)
    print('Received n2 =', decrypted_text)
    print('Message Authentic')
else:
    print('Message found not Authentic')
    sys.exit(1)
print()
print('Connection Established!')

Waiting for connection
CONNECTION FROM: ('127.0.0.1', 38462)

Message from Client 1(after decryption) :  client1||1635253162.6156664

connected succcessfully to Public-Key Authority

Request sent to server :  client1||1635253162.6205213

Response received from server(after decryption) 
 :  client1||1635253162.6156664


Public key of requested client : [5, 161]

Sent Time Stamp T2:  1635253162.6205213

Recieved Time Stamp T2:  1635253162.6223507

Message sending from Client 2(this client) to Client 1 :  1635253162.6156664||1635253162.622589
connected succcessfully to client1

Client 1 response :  1635253162.622589
Original n2 = 1635253162.622589
Received n2 = 1635253162.622589
Message Authentic

Connection Established!


In [8]:
count=1
while(count<=3):
    #Receiving Messages
    msg = s.recv(1024)
    print('Client 1 says(Encrypted Message): ', msg.decode())
    decrypted_text = decrypt_data(privateKey_client2, msg.decode())
    print('Client 1 says : ', decrypted_text)
    print()
    
    #Sending Messages
    message_reply = 'Got-it '+str(count)
    print('Client 2(this client) :', message_reply)
    encrypted_text = encrypt_data(requested_public_key, message_reply)
    print('Client 2 Encrypted Message(this Client): ', encrypted_text)
    print()
    s.send(encrypted_text.encode())
    
    count+=1
    
s.close()

Client 1 says(Encrypted Message):  YÑy
Client 1 says :  Hi 1

Client 2(this client) : Got-it 1
Client 2 Encrypted Message(this Client):  N"]Dw]di

Client 1 says(Encrypted Message):  YÑ!
Client 1 says :  Hi 2

Client 2(this client) : Got-it 2
Client 2 Encrypted Message(this Client):  N"]Dw]d

Client 1 says(Encrypted Message):  YÑ3
Client 1 says :  Hi 3

Client 2(this client) : Got-it 3
Client 2 Encrypted Message(this Client):  N"]Dw]d

